데이터 : data_atype.zip (수업노트에서 다운로드)

## 머신러닝
- 문제정의, 라이브러리/데이터 불러오기
- 탐색적 데이터 분석 (EDA)
- 데이터 전처리
- 피처엔지니어링
- (Train/Validation 나누기)
- 모델 선택/훈련/평가/최적화
- 예측
- (csv 생성)

## 1. 베이스라인
- 문제정의, 라이브러리 및 데이터 불러오기
- 데이터 전처리 (단순 일괄 처리)
- 모델 선택, 훈련
- 평가

## 2. 베이스라인
- 훈련/검증용 데이터 분리
- 모델 선택, 훈련
    - 의사결정나무
    - 랜덤포레스트
    - XGBoost
- 평가

## 문제1
- "<= 50K -> 0"
- "> 50K -> 1"
- 평가: 정확도

In [69]:
# 라이브러리 및 데이터 불러오기
import pandas as pd

X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [70]:
# 데이터 크기
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(29304, 15)
(29304, 2)
(3257, 15)


In [71]:
# 데이터 샘플
print(X_train.head(2))
print("-----")
print(y_train.head(2))
print("-----")
print(X_test.head(2))

      id   age  workclass  fnlwgt     education  education.num  \
0   3331  34.0  State-gov  177331  Some-college             10   
1  19749  58.0    Private  290661       HS-grad              9   

       marital.status      occupation relationship   race   sex  capital.gain  \
0  Married-civ-spouse  Prof-specialty      Husband  Black  Male          4386   
1  Married-civ-spouse    Craft-repair      Husband  White  Male             0   

   capital.loss  hours.per.week native.country  
0             0            40.0  United-States  
1             0            40.0  United-States  
-----
      id income
0   3331   >50K
1  19749  <=50K
-----
      id   age  workclass  fnlwgt     education  education.num  \
0  11574  39.0  State-gov  114055     Bachelors             13   
1  15847  38.0    Private  254114  Some-college             10   

          marital.status       occupation   relationship   race     sex  \
0          Never-married  Exec-managerial  Not-in-family  White  Female   
1

In [72]:
# 타겟 수 확인
y_train.value_counts(['income'])

income
<=50K     22263
>50K       7041
Name: count, dtype: int64

In [73]:
y_train['income'].value_counts()

income
<=50K    22263
>50K      7041
Name: count, dtype: int64

In [74]:
# type확인
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 3.4+ MB


In [75]:
# 수치형 데이터
cols = list(X_train.columns[X_train.dtypes!=object][1:])
X_train[cols].head()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0
2,48.0,125933,10,0,1669,38.0
3,58.0,100313,10,0,1902,40.0
4,41.0,195661,10,0,0,54.0


In [76]:
# 수치형 데이처 통계
X_train[cols].describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29292.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29291.000000
mean,38.553223,1.897488e+05,10.080842,1093.858722,86.744506,40.434229
std,13.628811,1.055250e+05,2.570824,7477.435640,401.518928,12.324036
min,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [77]:
# 결측값
X_train[cols].isnull().sum()

age               12
fnlwgt             0
education.num      0
capital.gain       0
capital.loss       0
hours.per.week    13
dtype: int64

In [78]:
# 간단한 결측치 처리
X_train = X_train.fillna(0)

In [79]:
# test도 채워줘야 함!!
X_test = X_test.fillna(0)

In [80]:
# 결측치 확인
print(X_train[cols].isnull().sum())
print(X_test[cols].isnull().sum())

age               0
fnlwgt            0
education.num     0
capital.gain      0
capital.loss      0
hours.per.week    0
dtype: int64
age               0
fnlwgt            0
education.num     0
capital.gain      0
capital.loss      0
hours.per.week    0
dtype: int64


In [ ]:
# 베이스 라인에서는 그외 전처리 및 피처 엔지니어링 생략

In [ ]:
# Model에 데이터를 넣기 위해서는 모든 값을 숫자로 변경해야 함
# target값 변경
# <=50K -> 0
# >50K -> 1

dict = {
    '<=50K': 0,
    '>50K': 1
}

y_train['income'] = y_train['income'].map(dict)

In [87]:
y = (y_train['income'] == '>50K').astype(int)
y[:3]

0    1
1    0
2    0
Name: income, dtype: int64

In [85]:
# 데이터 확인
print(X_train[cols])
print(y.head())

        age  fnlwgt  education.num  capital.gain  capital.loss  hours.per.week
0      34.0  177331             10          4386             0            40.0
1      58.0  290661              9             0             0            40.0
2      48.0  125933             10             0          1669            38.0
3      58.0  100313             10             0          1902            40.0
4      41.0  195661             10             0             0            54.0
...     ...     ...            ...           ...           ...             ...
29299  28.0   47168              6             0             0            40.0
29300  44.0  231793             16             0             0            38.0
29301  41.0  201435              9             0             0            40.0
29302  43.0  137722              9             0             0            40.0
29303  22.0  406978             13             0             0            40.0

[29304 rows x 6 columns]
0    1
1    0
2    0
3    

## 머신러닝 모델
- 분류 : 이진분류 문제, 0 또는 1 선택

1. 모델 선언
2. 학습 : fit()
3. 예측 : predict()

In [90]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier

# 모델 선언
model = RandomForestClassifier()

# 학습
model.fit(X_train[cols], y)

# 예측
pred = model.predict(X_test[cols])

In [92]:
pred[:50]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1])

In [99]:
# 데이터 크기 확인
print(len(pred))
print(len(y_train))
print(len(X_test))

3257
29304
3257


In [102]:
# 예측 및 csv 파일 생성
submit = pd.DataFrame(
        {
            'id': X_test['id'],
            'income': pred
        }
    )

submit.to_csv('0000.csv', index=False)

In [107]:
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('y_test.csv')
ans = (y_test['income'] == '>50K').astype(int)
# accuracy_score(실제값, 예측값)
accuracy_score(ans, pred)

0.8111759287688056

In [106]:
y_test

,id,income
0,11574,<=50K
1,15847,<=50K
2,17655,<=50K
3,19790,>50K
4,31812,<=50K
...,...,...
3252,20589,<=50K
3253,5668,<=50K
3254,27652,<=50K
3255,14735,<=50K


In [ ]:
# 평가 (수험자는 알 수 없는 부분임) accuracy
from sklearn.metrics import accuracy_score
y_test = pd.read_csv("y_test.csv")
ans = (y_test['income'] == '>50K').astype(int)
accuracy_score(ans, pred)

In [ ]:
y_test

## 문제2
- "<= 50K -> 0"
- "> 50K -> 1"
- 평가: roc_auc
- 예측 해야할 값 : 확률

## 검증용 데이터 분리
- 실제값으로 평가할 수 없기 때문에 검증용 데이터를 분리

In [112]:
# 학습용 데이터와 검증용 데이터로 구분
from sklearn.model_selection import train_test_split

y = (y_train['income'] == '>50K').astype(int)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size=0.1, random_state=123)

In [113]:
# 데이터 크기
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((26373, 15), (2931, 15), (26373,), (2931,))

In [114]:
y_tr

4245     1
22983    1
19565    0
4404     0
2282     0
        ..
28636    1
17730    1
28030    0
15725    0
19966    0
Name: income, Length: 26373, dtype: int64

In [127]:
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

model.fit(X_tr[cols], y_tr)
pred = model.predict_proba(X_val[cols]) # 확률값을 예측해야하기 때문에 predict_proba()

In [117]:
pred[:10] # 100%, 0% 확률

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [128]:
# roc_auc로 평가
from sklearn.metrics import roc_auc_score

# roc_auc_score(실제, 예측)
roc_auc_score(y_val, pred[:, 1])

0.6808862590712896

In [129]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_tr[cols], y_tr)
pred = model.predict_proba(X_val[cols])

In [130]:
pred[:10]

array([[0.88, 0.12],
       [0.88, 0.12],
       [0.82, 0.18],
       [0.69, 0.31],
       [0.95, 0.05],
       [0.02, 0.98],
       [0.67, 0.33],
       [0.69, 0.31],
       [1.  , 0.  ],
       [0.99, 0.01]])

In [131]:
# roc_auc로 평가
from sklearn.metrics import roc_auc_score

# roc_auc_score(실제, 예측)
roc_auc_score(y_val, pred[:, 1])

0.8181401453403465

In [132]:
# XGBoost
from xgboost import XGBClassifier

model = XGBClassifier()

model.fit(X_tr[cols], y_tr)
pred = model.predict_proba(X_val[cols])

In [133]:
pred[:10]

array([[0.8104935 , 0.18950646],
       [0.79522777, 0.20477222],
       [0.77139926, 0.22860071],
       [0.70092857, 0.2990714 ],
       [0.99256855, 0.00743146],
       [0.04835337, 0.9516466 ],
       [0.98159075, 0.01840924],
       [0.64929295, 0.35070705],
       [0.93548083, 0.0645192 ],
       [0.7940801 , 0.2059199 ]], dtype=float32)

In [134]:
# 0일 확률과 1일 확률을 선택해서 구해야함
# 상위 소득일 확률
pred[:, 1]

array([0.18950646, 0.20477222, 0.22860071, ..., 0.01990365, 0.08538244,
       0.38902074], dtype=float32)

In [135]:
# roc_auc로 평가
from sklearn.metrics import roc_auc_score

# roc_auc_score(실제, 예측)
roc_auc_score(y_val, pred[:, 1])

0.87132093205551

In [137]:
# 평가 데이터로 예측 및 csv파일 생성
pred = model.predict_proba(X_test[cols])

submit = pd.DataFrame(
    {
        'id': X_test['id'],
        'income': pred[:, 1]
    }
)

submit.to_csv('1111.csv', index=False)

# 평가
- 수험자는 알 수 없는 영역임

In [138]:
from sklearn.metrics import roc_auc_score
y_test = pd.read_csv("y_test.csv")
ans = (y_test['income'] != '<=50K').astype(int)
roc_auc_score(ans, pred[:,1])

0.8810225885225886